In [1]:
                                                                                                                                                                                                                    require 'cunn';
require 'nn';
require 'paths'
local matio = require 'matio'
local tensor = matio.load('CIFAR2_reshaped.mat')
test_X = tensor.test_X

nRepeat = 4
labelLength = nRepeat * 2
numberTR0 = 5000
numberTR1 = 5000
numberTe0 = 500
numberTe1 = 500
label0 = torch.cat(torch.ones(nRepeat), torch.zeros(nRepeat),1)
label1 = torch.cat(torch.zeros(nRepeat), torch.ones(nRepeat),1)
label0 = label0:cuda()
label1 = label1:cuda()

train_Y = torch.cat(torch.repeatTensor(label0, numberTR0, 1), torch.repeatTensor(label1, numberTR1, 1), 1)
test_Y = torch.cat(torch.repeatTensor(label0, numberTe0, 1), torch.repeatTensor(label1, numberTe1, 1), 1)

    tensors = {}
tensors.data = tensor.train_X
tensors.label = train_Y

--print(label0)
--print(train_Y[5001])
--print(torch.type(trainset))

In [2]:
-- ignore setmetatable for now, it is a feature beyond the scope of this tutorial. It sets the index operator.
setmetatable(tensors, 
    {__index = function(t, i) 
                    return {t.data[i], t.label[i]} 
                end}
);
tensors.data = tensors.data:double() -- convert the data from a ByteTensor to a DoubleTensor.

function tensors:size() 
    return self.data:size(1) 
end


--change to cuda after setmetatable. unknown reason
tensors.data = tensors.data:cuda()
tensors.label = tensors.label:cuda()


In [3]:
mean = {} -- store the mean, to normalize the test set in the future
stdv  = {} -- store the standard-deviation for the future
for i=1,3 do -- over each image channel
    mean[i] = tensors.data[{ {}, {i}, {}, {}  }]:mean() -- mean estimation
    print('Channel ' .. i .. ', Mean: ' .. mean[i])
    tensors.data[{ {}, {i}, {}, {}  }]:add(-mean[i]) -- mean subtraction, :add(value) add value to all elements in place.
    
    stdv[i] = tensors.data[{ {}, {i}, {}, {}  }]:std() -- std estimation
    print('Channel ' .. i .. ', Standard Deviation: ' .. stdv[i])
    tensors.data[{ {}, {i}, {}, {}  }]:div(stdv[i]) -- std scaling :div(value) divides all elements by value in place.
end

Channel 1, Mean: 122.4553527832	


Channel 1, Standard Deviation: 63.385623931885	


Channel 2, Mean: 120.61581420898	


Channel 2, Standard Deviation: 62.504470825195	


Channel 3, Mean: 111.08353424072	


In [4]:
net = nn.Sequential()
net:add(nn.SpatialZeroPadding(2,2,2,2))
net:add(nn.SpatialConvolution(3, 32, 5, 5)) -- 3 input image channels, 6 output channels, 5x5 convolution kernel
net:add(nn.Dropout(0.4))
net:add(nn.ReLU())                       -- non-linearity, think of this as sigmoid function 
net:add(nn.SpatialMaxPooling(2,2,2,2))     -- A max-pooling operation that looks at 2x2 windows and finds the max.
net:add(nn.SpatialZeroPadding(2,2,2,2))
net:add(nn.SpatialConvolution(32, 32, 5, 5)) --repeat convolution
net:add(nn.Dropout(0.4))
net:add(nn.ReLU())                       -- non-linearity 
net:add(nn.SpatialMaxPooling(2,2,2,2))
net:add(nn.SpatialZeroPadding(2,2,2,2))
net:add(nn.SpatialConvolution(32, 64, 5, 5)) --repeat convolution
net:add(nn.Dropout(0.4))
net:add(nn.ReLU())                       -- non-linearity 
net:add(nn.SpatialMaxPooling(2,2,2,2))
net:add(nn.View(64*4*4))                    -- reshapes from a 3D tensor of 16x5x5 into 1D tensor of 16*5*5
net:add(nn.Linear(64*4*4, 120))             -- fully connected layer (matrix multiplication between input and weights), neural network, full connection.
net:add(nn.ReLU())                       -- non-linearity 
net:add(nn.Linear(120, 84))
net:add(nn.ReLU())                       -- non-linearity 
net:add(nn.Dropout(0.5))
net:add(nn.Linear(84, labelLength))                   -- 10 is the number of outputs of the network (in this case, 10 digits)
net:add(nn.Sigmoid()) 
--net:add(nn.LogSoftMax())                     -- converts the output to a log-probability. Useful for classification problems
net = net:cuda()

In [5]:
criterion = nn.MSECriterion()
criterion = criterion:cuda()
trainer = nn.StochasticGradient(net, criterion)
trainer.learningRate = 0.0005
trainer.maxIteration = 500 -- just do 5 epochs of training.

In [6]:
trainer:train(tensors)

# StochasticGradient: training	


# current error = 0.24919563367069	


# current error = 0.2429745572269	


# current error = 0.21412023772264	


# current error = 0.12969063240561	


...

# current error = 0.0011294055006434	


# current error = 0.00091263403078633	


# current error = 0.00074919558177508	


# current error = 0.00061926874123782	


# current error = 0.00054060698694621	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 0.00054060698694621	



In [7]:
test_X = test_X:double()   -- convert from Byte tensor to Double tensor
for i=1,3 do -- over each image channel
    tensors.data[{ {}, {i}, {}, {}  }]:add(-mean[i]) -- mean subtraction    
    tensors.data[{ {}, {i}, {}, {}  }]:div(stdv[i]) -- std scaling
end

In [8]:
correct = 0
for i=1,numberTe0 do
    local prediction = net:forward(test_X[i])
    if i == 1 then print(prediction)end
    if (prediction - label0)*(prediction - label0)< (prediction - label1) * (prediction -label1) then
        correct = correct + 1
    end
end

for i=numberTe0+1,numberTe0 + numberTe1 do
    local prediction = net:forward(test_X[i])
    if i == 1000 then print(prediction)end
    if (prediction - label1)*(prediction - label1) < (prediction - label0)*(prediction - label0) then
        correct = correct + 1
    end
end

 1
 1
 1
 1
 0
 0
 0
 0
[torch.CudaTensor of size 8]



 0.0000
 0.0000
 0.0000
 0.0000
 1.0000
 1.0000
 1.0000
 1.0000
[torch.CudaTensor of size 8]



In [9]:
print(correct, 100*correct/(numberTe0+numberTe1) .. ' % ')

In [10]:
print(net.modules[22].weight)

751	75.1 % 	


Columns 1 to 10
 0.1179 -0.0965  0.0270 -0.0548 -0.0553  0.1322 -0.0333 -0.1513  0.1607 -0.0903
 0.1139 -0.1529 -0.0082  0.0652  0.0674  0.1491 -0.1574 -0.0979  0.1598 -0.1473
 0.0711 -0.1164 -0.0731 -0.0821  0.0409  0.1294 -0.0910 -0.1107  0.1210 -0.1693
 0.0700 -0.1495  0.0921  0.0006 -0.0422  0.1480 -0.1049 -0.1501  0.1616 -0.1523
-0.1048  0.1301  0.0800 -0.1051 -0.0677 -0.1271  0.0016  0.0369 -0.1023  0.1135
-0.1146  0.0907  0.0460  0.0885  0.0191 -0.1491  0.0213  0.0772 -0.1141  0.1504
-0.0793  0.1684  0.0666  0.0844 -0.0711 -0.1437  0.0902  0.1477 -0.1409  0.1578
-0.0701  0.1407  0.1079  0.0667 -0.1227 -0.1523  0.1241  0.1568 -0.0934  0.1470

Columns 11 to 20
-0.0162 -0.0906  0.1243  0.1103 -0.1229 -0.1649 -0.0375  0.1565  0.1450  0.0261
-0.1066  0.0535  0.1564  0.1171 -0.0234 -0.1468 -0.0058  0.1536  0.1240 -0.0397
 0.1057  0.0663  0.1049  0.1499 -0.1291 -0.1283 -0.0902  0.0536  0.1159 -0.0048
 0.0920  0.0503  0.1376  0.1330 -0.0195 -0.1027 -0.0672  0.0914  0.1362  0.1122
 0.013

645  0.0122 -0.0891  0.1453
 0.1341 -0.1245 -0.0322  0.1134 -0.0258 -0.0101 -0.0068 -0.0868 -0.0954  0.1582
 0.1447 -0.1670  0.0690  0.1082 -0.0538 -0.0060 -0.1371 -0.0931 -0.0881  0.1386
-0.1394  0.1551 -0.0027 -0.0991  0.0485  0.0732  0.0683  0.1411 -0.0901 -0.1648
-0.1289  0.1500 -0.1005 -0.1079  0.0904  0.1000  0.0162  0.0109 -0.0884 -0.1017
-0.1592  0.1298  0.0828 -0.0426 -0.0285  0.0700  0.1552  0.0504  0.0068 -0.1221
-0.1321  0.1432  0.0912 -0.1178 -0.0858 -0.0446  0.0800  0.1441 -0.0657 -0.1366

Columns 71 to 80
-0.1303  0.1297 -0.1346  0.0304 -0.0936 -0.0083 -0.0604 -0.1707 -0.0289  0.1184
-0.1617  0.0190 -0.0928  0.0357  0.0736 -0.1199 -0.1297 -0.1362  0.0885  0.1317
-0.0860  0.0887 -0.1410  0.0728 -0.0277 -0.0718 -0.0782 -0.1560 -0.0262  0.1114
-0.1227  0.0699 -0.0093  0.1268  0.0566 -0.0262 -0.1018 -0.1351  0.0638  0.0835
 0.1737 -0.1279  0.0580 -0.0063  0.0480  0.0162  0.0537  0.0768  0.0751 -0.0118
 0.1347 -0.1203  0.1141 -0.1213  0.0714 -0.0351  0.0771  0.1749 -0.0129 -0

* Build crazy graphs of networks: https://github.com/torch/nngraph
* Train on imagenet with multiple GPUs: https://github.com/soumith/imagenet-multiGPU.torch
* Train recurrent networks with LSTM on text: https://github.com/wojzaremba/lstm

* More demos and tutorials: https://github.com/torch/torch7/wiki/Cheatsheet

* Chat with developers of Torch: http://gitter.im/torch/torch7
* Ask for help: http://groups.google.com/forum/#!forum/torch7
